### https://www.census.gov/data/developers/data-sets.html

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
import json
import requests

path=os.getcwd()

### check variables

In [2]:
survey=[
        ['ASC_5year_subject','https://api.census.gov/data/2019/acs/acs5/subject/']
       ]

In [3]:
def clean_names(x):
    p=x.find('<th>')
    if p>-1:
        x=x[(p+4):]
    p=x.find('</th>')
    if p>-1:
        x=x[:p]
    return(x)

def clean_tbl(x):
    if x.find('N/A')>-1:
        return('N/A')
    else:
        p=x.find('>')
        if p>-1:
            x=x[(p+1):]
        p=x.find('<')
        if p>-1:
            x=x[:p]
        return(x)

In [4]:
variables=pd.DataFrame()

for s in survey:
    print(s[0])
    if s[1]!='':
        r=requests.get(s[1]+'variables.html')
        r=r.text
        
        p=r.find('<table>')
        p=r.find('<table>')
        r=r[p:]
        p1=r.find('<thead>')
        r=r[p1:]
        r=r.split('\n<tr>\n<td>')
        
        cols=[clean_names(x) for x in r[0].split('</th><th>')]
        y=[[clean_tbl(y) for y in x.split('</td><td>')] for x in r[1:]]
        
        v=pd.DataFrame(y,columns=cols)
        v['purpose']=s[0]
        
        variables=pd.concat([variables,v])

ASC_5year_subject


In [5]:
kmeans_varaibles=pd.read_csv(path+'/kmeans_variables.csv')
kmeans_varaibles[:3]

,Category,Name,Label,Concept,Required,Attributes,Limit,Predicate Type,Group
0,Cultural,S0102_C01_061E,"Estimate!!Total!!PLACE OF BIRTH, NATIVITY AND ...",POPULATION 60 YEARS AND OVER IN THE UNITED STATES,not required,"S0102_C01_061EA, S0102_C01_061M, S0102_C01_061MA",0,float,S0102
1,Cultural,S0102_C01_065E,Estimate!!Total!!LANGUAGE SPOKEN AT HOME AND A...,POPULATION 60 YEARS AND OVER IN THE UNITED STATES,not required,"S0102_C01_065EA, S0102_C01_065M, S0102_C01_065MA",0,float,S0102
2,Cultural,S1601_C01_004E,Estimate!!Total!!Population 5 years and over!!...,LANGUAGE SPOKEN AT HOME,not required,"S1601_C01_004EA, S1601_C01_004M, S1601_C01_004MA",0,int,S1601


In [6]:
kmeans_varaibles.shape

(106, 9)

## get rid of percent variables

In [7]:
kmeans_variables=pd.read_csv(path+'/kmeans_variables.csv')
variables=pd.read_csv(path+'/variables.csv')
variables=variables[variables['purpose']=='ASC_5year_subject']

In [8]:
TF=kmeans_variables['Label'].str.lower().str.find('percent ')>-1
pct=kmeans_variables[TF]
pct.shape

(19, 9)

In [9]:
pct=pct[['Category','Label']].rename(columns={'Label':'Label2'})
pct['Label']=pct['Label2'].str.replace('Percent ','')
pct.shape

(19, 3)

In [10]:
non_pct=variables.merge(pct[['Category','Label']],how='inner',on='Label')
non_pct.shape

(19, 10)

In [11]:
kmeans_variables=pd.concat([kmeans_variables[~TF],non_pct])

In [12]:
TF=kmeans_variables['Label'].str.lower().str.find('percent ')>-1
TF.unique()

array([False])

In [13]:
kmeans_variables.to_csv(path+'/kmeans_variables2.csv',index=False)

### get data

In [14]:
v=list(kmeans_variables['Name'])
if 'S0101_C01_001E' not in v:
    v=['S0101_C01_001E'] +v
len(v)

107

In [15]:
url='https://api.census.gov/data/2019/acs/acs5/subject'
columns='NAME,S0101_C01_001E,'+','.join(list(kmeans_varaibles['Name'])) #'NAME,group(S0102)'
region='zip%20code%20tabulation%20area:*' #'county' #'state' # 'us:1'
key='569a375ff2e18899eed44d35de29dac9419e938b'

In [16]:
raw=[]
i=1
while 49*(i-1)<len(v):
    columns=','.join(v[49*(i-1):(49*i)])
    link=url+'?get='+columns+'&for='+region +'&key='+key
    with urlopen(link) as x:
         data = x.read().decode('utf-8')
            
    p=data.find(']')
    df=pd.read_json('['+data[(p+3):])
    df.columns=data[2:(p)].replace('"','').split(',')
    
    if type(raw)==list:
        raw=df
    else:
        raw=raw.merge(df,how='outer',on=['state','zip code tabulation area'])

    i=i+1

In [17]:
[x for x in v if x not in list(raw.columns)]

[]

In [18]:
raw.shape

(33120, 109)

In [19]:
raw.to_csv(path+'/01_raw.csv',index=False)